
# 🇨🇱 Simulador FES — Plan de Reorganización y Condonación de Deudas Educativas

Este *notebook* implementa un simulador basado en el **Proyecto de Ley del FES y Plan de Reorganización y Condonación** (Boletín N°17169-04, mayo 2025).  
Incluye:
- **Actualización automática de UF y UTM** (con opción de ingreso manual si falla el servicio).
- **Condonación inicial** según perfil y avance de pago.
- **Opción de condonación adicional (25%)** por **pago anticipado en 12 cuotas**.
- **Cálculo del pago mensual contingente al ingreso (FES)** usando **tramos en UTA** y el tope global **7%/8%**.
- **Condonación progresiva mensual** (si el pago FES es menor que la cuota recalculada).
- **Comparativa** con el método vigente (ej.: cuota actual; y, si corresponde, referencia a **FSCU 5%** del ingreso).

> **Nota:** Para usarlo en Google Colab, simplemente sube este archivo y ejecútalo de arriba hacia abajo. Si deseas, conecta tu cuenta de Google Drive para guardar resultados.


## 1) Indicadores (UF, UTM) — actualización automática con *fallback* manual

In [ ]:

#@title Descarga de UF/UTM y configuración inicial
import json, math, datetime, sys
from dataclasses import dataclass

try:
    import requests  # Colab suele traerlo instalado
except ImportError:
    !pip -q install requests
    import requests

def _fetch_indicator(indicador: str) -> float:
    '''
    Intenta obtener el valor CLP del indicador (uf, utm) desde mindicador.cl.
    Si falla, levanta una excepción (manejada abajo).
    '''
    # mindicador.cl expone /api/uf y /api/utm, ambos retornan una serie con el último valor al índice 0
    url = f"https://mindicador.cl/api/{indicador}"
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    data = r.json()
    if isinstance(data, dict) and "serie" in data and data["serie"]:
        return float(data["serie"][0]["valor"])
    # Respaldo en caso de formato alternativo
    if isinstance(data, dict) and "valor" in data:
        return float(data["valor"])
    raise RuntimeError(f"Formato inesperado desde {url}")

def get_indicadores(auto: bool=True,
                    uf_manual: float=None,
                    utm_manual: float=None) -> dict:
    '''
    Obtiene UF y UTM en CLP.
    - Si auto=True, intenta descargar ambos; si falla, usa manual (si viene) o pide ingreso.
    - Devuelve dict con: uf_clp, utm_clp, uta_clp (=12*UTM), fuente.
    '''
    fuente = "manual"
    uf_clp = None
    utm_clp = None

    if auto:
        try:
            uf_clp = _fetch_indicator("uf")
            utm_clp = _fetch_indicator("utm")
            fuente = "mindicador.cl"
        except Exception as e:
            print("⚠️ No se pudo obtener UF/UTM automáticamente:", e, file=sys.stderr)

    # Fallback a manual si fuese necesario
    if uf_clp is None:
        if uf_manual is None:
            uf_manual = float(input("Ingrese valor UF (CLP): ").strip())
        uf_clp = uf_manual

    if utm_clp is None:
        if utm_manual is None:
            utm_manual = float(input("Ingrese valor UTM (CLP): ").strip())
        utm_clp = utm_manual

    uta_clp = 12.0 * utm_clp  # 1 UTA = 12 UTM
    return {
        "uf_clp": uf_clp,
        "utm_clp": utm_clp,
        "uta_clp": uta_clp,
        "fuente": fuente,
        "fecha": datetime.date.today().isoformat()
    }

# Ejecuta la obtención automática por defecto (puedes ajustar uf_manual/utm_manual si lo deseas)
INDICADORES = get_indicadores(auto=True)
print(f"UF:  ${INDICADORES['uf_clp']:,.0f}  |  UTM: ${INDICADORES['utm_clp']:,.0f}  |  UTA: ${INDICADORES['uta_clp']:,.0f}  (fuente: {INDICADORES['fuente']}, {INDICADORES['fecha']})")


## 2) Fórmulas del plan (condonaciones y pago contingente FES)

In [ ]:

#@title Fórmulas principales

from typing import Tuple

PERFILES_BASE_UF = {
    # Base de condonación inicial por perfil (en UF)
    # Desertó al día / en mora; Egresó al día / en mora
    "Desertó, al día": 60.0,
    "Desertó, con mora": 30.0,
    "Egresó, al día": 40.0,
    "Egresó, con mora": 20.0,
}

def condonacion_inicial_uf(perfil: str, cuotas_pagadas: int, cuotas_totales: int,
                           deuda_uf: float) -> Tuple[float, float]:
    """
    Cálculo de condonación inicial en UF según fórmula oficial:
      condonación_inicial = Base(perfil) * (1 + cuotas_pagadas / cuotas_totales)
    Luego se limita a no exceder la deuda vigente.
    Devuelve (condonación_uf, nueva_deuda_uf).
    """
    if perfil not in PERFILES_BASE_UF:
        raise ValueError(f"Perfil inválido: {perfil}")
    if cuotas_totales <= 0:
        raise ValueError("cuotas_totales debe ser > 0")
    cuotas_pagadas = max(0, min(cuotas_pagadas, cuotas_totales))

    base = PERFILES_BASE_UF[perfil]
    cond_uf = base * (1.0 + (cuotas_pagadas / float(cuotas_totales)))
    cond_uf = min(cond_uf, max(0.0, deuda_uf))
    nueva_deuda_uf = max(0.0, deuda_uf - cond_uf)
    return cond_uf, nueva_deuda_uf

def condonacion_adicional_pago_anticipado_uf(nueva_deuda_uf: float) -> Tuple[float, float]:
    """
    Si la persona opta por pagar la NUEVA deuda (post condonación inicial) en 12 cuotas,
    recibe una condonación adicional del 25% de ese nuevo saldo.
    Retorna (condonacion_adicional_uf, cuota_mensual_uf_en_12).
    """
    cond_extra = 0.25 * max(0.0, nueva_deuda_uf)
    saldo_a_pagar = max(0.0, nueva_deuda_uf - cond_extra)
    cuota_mensual_12 = saldo_a_pagar / 12.0 if saldo_a_pagar > 0 else 0.0
    return cond_extra, cuota_mensual_12

def pago_mensual_fes(ingreso_mensual_clp: float, utm_clp: float) -> float:
    """
    Calcula el PAGO MENSUAL bajo el esquema FES.
    - Se determina el ingreso anual (12x), se convierte a UTA (UTA=12*UTM).
    - Tramos marginales: exento hasta 7.5 UTA; 13% entre 7.5–11.2 UTA; 15% sobre 11.2 UTA.
    - Tope global: 7% del ingreso si <= 45 UTA; 8% si > 45 UTA.
    - Se calcula pago anual y se divide por 12 para obtener el pago mensual.
    """
    ingreso_mensual = max(0.0, float(ingreso_mensual_clp))
    ingreso_anual = ingreso_mensual * 12.0
    uta_clp = 12.0 * utm_clp
    if uta_clp <= 0:
        raise ValueError("UTM no válida — UTA resultó <= 0")

    ingreso_en_uta = ingreso_anual / uta_clp

    # Componentes marginales
    tramo_exento = 7.5
    tramo_2 = 11.2

    base_2_uta = max(0.0, min(ingreso_en_uta, tramo_2) - tramo_exento)  # hasta 11.2
    base_3_uta = max(0.0, ingreso_en_uta - tramo_2)                     # sobre 11.2

    pago_anual_marginal = (0.13 * base_2_uta + 0.15 * base_3_uta) * uta_clp

    # Tope global 7%/8%
    if ingreso_en_uta <= 45.0:
        tope_anual = 0.07 * ingreso_anual
    else:
        tope_anual = 0.08 * ingreso_anual

    pago_anual = min(pago_anual_marginal, tope_anual)
    return pago_anual / 12.0

def cuota_recalculada(cuota_original_mensual_clp: float,
                      condonacion_inicial_clp: float,
                      cuotas_totales: int,
                      cuotas_pagadas: int) -> float:
    """
    Recalcula la cuota mensual post condonación inicial (si se mantiene el plan en cuotas).
    Fórmula referencial: A = cuota_original - (condonacion_inicial / cuotas_pendientes).
    Donde cuotas_pendientes = max(1, cuotas_totales - cuotas_pagadas).
    Se trunca a mínimo 0.
    """
    cuotas_pagadas = max(0, min(cuotas_pagadas, cuotas_totales))
    pendientes = max(1, int(cuotas_totales - cuotas_pagadas))
    ajuste = float(condonacion_inicial_clp) / float(pendientes)
    A = float(cuota_original_mensual_clp) - ajuste
    return max(0.0, A)

def comparativa_mensual(cuota_recalc_clp: float, pago_fes_clp: float) -> dict:
    """
    Si B (pago FES) < A (cuota recalculada), se paga B y se condona la diferencia (A-B) cada mes.
    """
    A = float(cuota_recalc_clp)
    B = float(pago_fes_clp)
    pago_efectivo = min(A, B)
    condonacion_mensual = max(0.0, A - B)
    return {"pago_efectivo": pago_efectivo, "condonacion_mensual": condonacion_mensual}

def pago_vigente_fscu_5pct(ingreso_mensual_clp: float) -> float:
    """
    Referencia del régimen FSCU vigente: pago contingente al 5% del ingreso.
    (Se utiliza sólo para comparativa; en el Plan se aplica FES si resulta <= a este).
    """
    return 0.05 * max(0.0, float(ingreso_mensual_clp))


## 3) Interfaz interactiva

In [ ]:

#@title Interfaz
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

style = {'description_width': 'initial'}

tipo_credito = widgets.Dropdown(
    options=["CAE", "Fondo Solidario (FSCU)"],
    value="CAE",
    description="Tipo de crédito:",
    style=style
)

deuda_actual_clp = widgets.FloatText(
    value=6_000_000.0,
    description="Deuda vigente (CLP):",
    style=style
)

perfil = widgets.Dropdown(
    options=list(PERFILES_BASE_UF.keys()),
    value="Egresó, al día",
    description="Perfil para condonación inicial:",
    style=style
)

cuotas_totales = widgets.IntText(
    value=240,
    description="Cuotas totales (p. ej. 240):",
    style=style
)

anios_pagados = widgets.IntText(
    value=10,
    description="Años pagados (x12 = cuotas):",
    style=style
)

cuota_actual = widgets.FloatText(
    value=75_000.0,
    description="Tu cuota mensual actual (CLP):",
    style=style
)

ingreso_mensual = widgets.FloatText(
    value=800_000.0,
    description="Ingreso bruto mensual (CLP):",
    style=style
)

usar_pago_anticipado = widgets.Checkbox(
    value=False,
    description="Optar por pago anticipado (12 cuotas, condonación adicional 25%)",
    style=style
)

btn = widgets.Button(description="Simular", button_style="success")
out = widgets.Output()

def _fmt(x): 
    try:
        return f"${x:,.0f}"
    except:
        return str(x)

def on_click(_):
    with out:
        clear_output(wait=True)
        try:
            uf = INDICADORES["uf_clp"]
            utm = INDICADORES["utm_clp"]
            uta = INDICADORES["uta_clp"]
        except KeyError:
            print("Error: no están disponibles UF/UTM. Vuelve a ejecutar la celda de indicadores.")
            return

        cuotas_pagadas = int(max(0, anios_pagados.value) * 12)
        deuda_uf = float(max(0.0, deuda_actual_clp.value) / uf)

        # 1) Condonación inicial
        cond_ini_uf, nueva_deuda_uf = condonacion_inicial_uf(
            perfil=perfil.value,
            cuotas_pagadas=cuotas_pagadas,
            cuotas_totales=max(1, int(cuotas_totales.value)),
            deuda_uf=deuda_uf
        )
        cond_ini_clp = cond_ini_uf * uf
        nueva_deuda_clp = nueva_deuda_uf * uf

        # 2) Pago anticipado (opcional)
        cond_extra_clp = 0.0
        cuota_12_clp = None
        if usar_pago_anticipado.value and nueva_deuda_uf > 0:
            cond_extra_uf, cuota_12_uf = condonacion_adicional_pago_anticipado_uf(nueva_deuda_uf)
            cond_extra_clp = cond_extra_uf * uf
            cuota_12_clp = cuota_12_uf * uf

        # 3) FES: pago mensual contingente al ingreso
        pago_fes_clp = pago_mensual_fes(ingreso_mensual.value, utm)

        # 4) Recalcular cuota mensual (plan en cuotas tras cond. inicial)
        A = cuota_recalculada(
            cuota_original_mensual_clp=cuota_actual.value,
            condonacion_inicial_clp=cond_ini_clp,
            cuotas_totales=max(1, int(cuotas_totales.value)),
            cuotas_pagadas=cuotas_pagadas
        )

        # 5) Condonación progresiva mensual si B < A
        comp = comparativa_mensual(A, pago_fes_clp)

        # 6) Comparativas con método vigente
        base_vigente = cuota_actual.value  # si CAE, se asume cuota actual informada
        fscu_5 = pago_vigente_fscu_5pct(ingreso_mensual.value) if "Fondo" in tipo_credito.value else None

        # Tabla resumen
        filas = [
            ["Valor UF", _fmt(uf)],
            ["Valor UTM", _fmt(utm)],
            ["Valor UTA (12 UTM)", _fmt(uta)],
            ["Condonación inicial (CLP)", _fmt(cond_ini_clp)],
            ["Nueva deuda post cond. inicial (CLP)", _fmt(nueva_deuda_clp)],
            ["Cuota recalculada A (CLP)", _fmt(A)],
            ["Pago FES B (CLP)", _fmt(pago_fes_clp)],
            ["Pago efectivo (min A,B)", _fmt(comp["pago_efectivo"])],
            ["Condonación mensual progresiva (A-B si B<A)", _fmt(comp["condonacion_mensual"])],
        ]

        if usar_pago_anticipado.value:
            filas += [
                ["Condonación adicional (25% de nueva deuda)", _fmt(cond_extra_clp)],
                ["Cuota mensual si pago en 12 (CLP)", _fmt(cuota_12_clp)],
            ]

        filas += [
            ["Tu cuota vigente informada (referencia)", _fmt(base_vigente)],
        ]
        if fscu_5 is not None:
            filas += [["Referencia régimen FSCU vigente (5% ingreso)", _fmt(fscu_5)]]

        df = pd.DataFrame(filas, columns=["Ítem", "Valor"])
        display(df)

        # Mensajes clave
        print("\n🔎 Lectura rápida:")
        print(f"- Con FES, tu pago mensual contingente estimado sería: { _fmt(pago_fes_clp) }")
        print(f"- Si te mantienes en cuotas, tu cuota recalculada sería: { _fmt(A) }")
        print(f"- Se aplicaría condonación mensual de: { _fmt(comp['condonacion_mensual']) } si el pago FES es menor que la cuota recalculada.")
        if usar_pago_anticipado.value:
            print(f"- Si eliges pago anticipado, tu cuota por 12 meses sería: { _fmt(cuota_12_clp) } (con condonación adicional 25%).")

btn.on_click(on_click)

display(tipo_credito, deuda_actual_clp, perfil, cuotas_totales, anios_pagados, cuota_actual, ingreso_mensual, usar_pago_anticipado, btn, out)


## 4) (Opcional) Pruebas rápidas

In [ ]:

#@title Validación mínima de fórmulas
uf = INDICADORES["uf_clp"]; utm = INDICADORES["utm_clp"]

# Ejemplo de la lámina: Egresó, al día; 10 años de 20 (120/240) -> 40 * (1 + 0.5) = 60 UF
cond_uf, _ = condonacion_inicial_uf("Egresó, al día", cuotas_pagadas=120, cuotas_totales=240, deuda_uf=1e9)
print("Ejemplo esperado ~60 UF:", round(cond_uf, 4))

# Pago FES a ~$500k mensual (cercano tramo exento): debiese estar muy cerca de 0
print("Pago FES con ingreso ~500k CLP:", round(pago_mensual_fes(500_000, utm)))
